In [61]:
import numpy as np
import pandas as pd
import keyring
import requests
import json

infoblox_username = "rswarnka";
infoblox_password = keyring.get_password("Infoblox", infoblox_username)

NUMBER_OF_NETWORKS = 50000

infoblox_url = "https://gnoc-ipam.capgemini.com/wapi/v2.7/network?_return_fields=network,network_view,comment&_return_fields=extattrs&_return_type=json&_max_results="+str(NUMBER_OF_NETWORKS)  #50000

session = requests.Session()
session.auth = (infoblox_username, infoblox_password)

auth = session.post('https://gnoc-ipam.capgemini.com/wapi/v2.7/', cert=())

response = session.get(infoblox_url)

json_response = json.loads(response.content)


PATH = "Input/gnoc_networks.json";

filepath = open(PATH, 'w')

json.dump(json_response, filepath)
          
filepath.close()


print("Total Time elapsed getting response: " + str(response.elapsed.total_seconds()) +" seconds")
print("Size of response in KiB: " + str(round(   len(response.content) / 1024.0, 2 ) ) +" KiB")



Total Time elapsed getting response: 65.517549 seconds
Size of response in KiB: 11262.7 KiB


# `Certificate verify failed`:

Reference: [Link](https://github.com/psf/requests/issues/4381#issuecomment-612830533)

If you are beginner to python `requests` module and want to do some stuffs that requires accessing secure sites, there is high chance that you will be doomed by this error - `Certificate verify failed` and like all beginner programmers you will be tempted to use `auth = session.post( mysecureurl, verify=false)`

But this is very bad practice and have been discouraged over a lots of SO posts, but still beginners misuse this because the error is so sucky to fix.  
Let me attempt to throw some light on this issue. 
Python(pip) and Conda and whatsoever python based software uses separate **certificate store** just like all browsers do. The Python Requests library uses its [own CA file](https://incognitjoe.github.io/adding-certs-to-requests.html) by default, or will use the certifi package's certificate bundle if installed. Also, pip does not use the system certs unlike the `curl` do.
Hence the for the `requests` you have to manually specify the certificate store through the conda or pip. 

Tldr;
1. Export all the `.cer` encoded certificate chain using browser as per [this amazing blog](http://blog.majcica.com/2016/12/27/installing-self-signed-certificates-into-git-cert-store/) shown over here. Note, that blog is not about conda certstore but git certstore and it only says to export the root, however I exported **all the certificate chains into separate files**.
2. Next, install `certifi` using command `pip install certifi`
3. Check the default path of the cert store of conda or python:

`import ssl`
`ssl.get_default_verify_paths()` OR 
`import certifi`
`certifi.where()`

4. Once you have located the default `cacert.pem` file, open this (preferably into Notepad++) and append all the certificate at the end of file. (Take care of certificate demarcation `-----BEGIN CERTIFICATE-----` and `-----END CERTIFICATE-----`). Save the file and that is it. 
Or if you are using conda use the conda commands: 
`conda config --set ssl_verify <pathToYourFile>.crt`
(I have noticed this command updates stuff in `C:\Users\johndoe\.condarc`)

5. Use the below code to verify: 
`import certifi`
`auth = session.post('https://mysecuresite.com/', cert=());`

Also, if you are on linux, you can export custom cacert into system-wide or user profile (`.bashrc` or `.bash_profile`) using [this link](https://stackoverflow.com/questions/38571099/how-can-i-set-the-certificates-in-curl).



In [33]:
import json
import jmespath

In [62]:
PATH = "Input/gnoc_networks.json"
 

json_file = open(PATH, mode='r', encoding='utf8')

data = json.load(json_file,encoding="utf8")

filepath.close()
    
_query  = """
                  [].{network:network, City: extattrs.City.value, 
                  Country:extattrs.Country.value, 
                  ATON_Building_ID:extattrs.ATON_Building_ID.value,  
                  CRES_Building_ID:extattrs.CRES_Building_ID.value, 
                  Building:extattrs.Building.value, 
                  File_Name:extattrs.File_Name.value, 
                  Location:extattrs.Location.value, 
                  Region:extattrs.Region.value, 
                  Site:extattrs.Site.value, 
                  Support_Group:extattrs.Support_Group.value, comments:comment }"""

path = jmespath.search(_query,data)




In [63]:
df = pd.DataFrame(path)

In [64]:
df.head()

,network,City,Country,ATON_Building_ID,CRES_Building_ID,Building,File_Name,Location,Region,Site,Support_Group,comments
0,10.100.131.0/27,Mumbai,India,394,IN041,Gigaplex,IN-NAV-AIROLIGIGAPLEX,AIROLI OTHERS,APAC,Mumbai,Group IT,ITICS VLAN-VLAN 5
1,10.100.131.160/27,Amsterdam,Netherlands,Datacenter,IN041,Datacenter,IN-NAV-AIROLIGIGAPLEX,DCA,Northern Europe,Mumbai,Group IT,None
2,10.100.132.0/22,Mumbai,India,394,IN041,Gigaplex,IN-NAV-AIROLIGIGAPLEX,AIROLI OTHERS,APAC,Mumbai,Group IT,IPT 7th Floor VLAN-VLAN 10
3,10.100.136.0/23,Mumbai,India,394,IN041,Gigaplex,IN-NAV-AIROLIGIGAPLEX,AIROLI OTHERS,APAC,Mumbai,Group IT,IPT 8th Floor VLAN-VLAN 80
4,10.100.138.0/23,Mumbai,India,394,IN041,Gigaplex,IN-NAV-AIROLIGIGAPLEX,AIROLI OTHERS,APAC,Mumbai,Group IT,IPT 6th Floor VLAN-VLAN 90


In [66]:
len(df)

16947

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16947 entries, 0 to 16946
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   network           16947 non-null  object
 1   City              15317 non-null  object
 2   Country           15340 non-null  object
 3   ATON_Building_ID  15212 non-null  object
 4   CRES_Building_ID  6632 non-null   object
 5   Building          15236 non-null  object
 6   File_Name         6594 non-null   object
 7   Location          15224 non-null  object
 8   Region            15628 non-null  object
 9   Site              3103 non-null   object
 10  Support_Group     9685 non-null   object
 11  comments          10574 non-null  object
dtypes: object(12)
memory usage: 1.6+ MB


In [58]:
df[df['City'].isna()]

,network,City,Country,ATON_Building_ID,CRES_Building_ID,Building,File_Name,Location,Region,Site,Support_Group,comments
37,10.48.147.96/27,None,None,None,None,None,None,None,None,Mumbai,None,None
42,10.48.177.16/28,None,None,None,None,None,None,None,None,Mumbai,None,None
43,10.48.182.128/26,None,None,None,None,None,None,None,None,Mumbai,None,None
44,10.48.190.0/27,None,None,None,None,None,None,None,None,Mumbai,None,None
45,10.48.64.0/28,None,None,None,None,None,None,None,None,Mumbai,None,None


In [67]:
df.to_csv('Input/gnoc_networks.csv', index=False, sep=';', encoding='utf-8')